In [4]:
import geopandas as gpd
from shapely import wkt

In [6]:
building = gpd.read_file("../../purple-hawk-data/building.csv")

In [7]:
building_essential_short = building.head(10).reindex(
    ["BIN", "CNSTRCT_YR", "HEIGHTROOF", "GROUNDELEV", "FEAT_CODE"], axis=1
)

In [8]:
building_tax_lot_short = building.head(10).reindex(["BIN", "MPLUTO_BBL"], axis=1)

In [18]:
building_wgs84_short = building.head(10).reindex(["BIN", "the_geom"], axis=1)

building_wgs84_short["geometry"] = building_wgs84_short.the_geom.apply(wkt.loads)
building_wgs84_short.drop("the_geom", axis=1, inplace=True)
building_wgs84_short = gpd.GeoDataFrame(
    building_wgs84_short, geometry="geometry", crs=4326
)

In [20]:
building_long_island_short = building_wgs84_short.to_crs(2263)

In [22]:
building_long_island_short.to_csv(
    "../../purple-hawk-data/building_long_island_short_v1.csv", index=False
)
building_wgs84_short.to_csv(
    "../../purple-hawk-data/building_wgs84_short_v1.csv", index=False
)
building_essential_short.to_csv(
    "../../purple-hawk-data/building_essentials_short_v1.csv", index=False
)
building_tax_lot_short.to_csv(
    "../../purple-hawk-data/building_tax_lot_short_v1.csv", index=False
)